# Density Plotting with Amplitude-Based Color Coding

This notebook tests the new amplitude-based color coding feature for density plots.

## Test Objectives
1. Load density data from cache/results (HDF5 files)
2. Extract probe signal amplitudes from raw data
3. Test `plot_density` with `color_by_amplitude=True`
4. Verify that amplitude values are correctly mapped to colors

## Data Sources
- Cache files: `ifi/results/*/shot_*.h5`
- Results files: `ifi/results/*/shot_*.h5`


In [1]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path().resolve().parent.parent
sys.path.insert(0, str(project_root))

import numpy as np
import pandas as pd
from argparse import Namespace
import matplotlib.pyplot as plt
from ifi.analysis.plots import Plotter
from ifi.utils.file_io import load_results_from_hdf5
from ifi.db_controller.nas_db import NAS_DB
from ifi.db_controller.vest_db import VEST_DB
from ifi.utils.common import LogManager

# Initialize logging with INFO level (not DEBUG)
# This sets the root logger level to INFO, preventing DEBUG messages from being displayed
LogManager(level="INFO")
logger = LogManager().get_logger(__name__)

print(f"Project root: {project_root}")
print(f"Python path: {sys.path[0]}")


Project root: C:\Users\dhkdw\Documents
Python path: C:\Users\dhkdw\Documents


## Step 1: Find Available HDF5 Results Files


In [ ]:
results_dir = project_root / "ifi" / "results"
h5_files = list(results_dir.glob("*/shot_*.h5")) + list(results_dir.glob("*/*.h5"))

print(f"Found {len(h5_files)} HDF5 result files:")
if len(h5_files) < 1:
    print("No HDF5 files found. Running main_analysis.py first.")
    try:
        nas_db = NAS_DB(config_path="ifi/config.ini")
        vest_db = VEST_DB(config_path="ifi/config.ini")
        from ifi.analysis.main_analysis import run_analysis
        from ifi.analysis.interactive_analysis import create_mock_args
        args = create_mock_args()
        args.query = ["45821"]
        args.density = True
        args.plot = False
        args.save_data = True
        args.save_plots = False
        args.scheduler = "single-threaded"

        results = run_analysis(
            query=args.query,
            args=args,
            nas_db=nas_db,
            vest_db=vest_db,
        )
    except Exception as e:
        print(f"Failed to run main_analysis.py: {e}")
        raise e
for h5_file in h5_files[:5]:  # Show first 5
    print(f"  - {h5_file.relative_to(project_root)}")

if len(h5_files) > 5:
    print(f"  ... and {len(h5_files) - 5} more")


Found 0 HDF5 result files:
No HDF5 files found. Running main_analysis.py first.


DEBUG    | 
[LOGS -UPDAT] Log level updated to: DEBUG

INFO     | [ANALY-RUN  ] Parsing Analysis Query
INFO     | [ANALY-RUN  ] Found 1 unique shot numbers: [45821]
INFO     | [ANALY-RUN  ] Found 0 unique file paths: []
INFO     | [NASDB-CONN ] Local NAS mount found at 'Z:\'. Using direct access.
INFO     | [NASDB-QFILE] Searching for files for query: [45821] in folders: ['6. user/Jongin Wang/IFO/Data', '6. user/Byun JunHyeok/IF'].
INFO     | [NASDB-QFILE] Found 3 files. Caching result.
INFO     | [ANALY-RUN  ] Interferometry (Shot #45821) params for 45821_056.csv: 
INFO     | [ANALY-RUN  ] CDM method, 94.0 GHz
INFO     | [ANALY-RUN  ] Interferometry (Shot #45821) params for 45821_789.csv: 
INFO     | [ANALY-RUN  ] CDM method, 94.0 GHz
INFO     | [ANALY-RUN  ] Interferometry (Shot #45821) params for 45821_ALL.csv: 
INFO     | [ANALY-RUN  ] CDM method, 282.0 GHz
INFO     | [ANALY-RUN  ] Grouped files into 1 shot(s).

INFO     | [ANALY-RUN  ] Loading VEST data for shots: [45821]
INFO    

## Step 2: Load Density Data from HDF5 File


In [3]:
# Select a test file (use first available)
if h5_files:
    test_h5_file = h5_files[0]
    print(f"Loading: {test_h5_file.name}")
    
    # Extract shot number from path
    shot_num = int(test_h5_file.parent.name) if test_h5_file.parent.name.isdigit() else 45821
    base_dir = test_h5_file.parent.parent
    
    # Load results
    results = load_results_from_hdf5(shot_num, base_dir=str(base_dir))
    
    if results:
        print(f"\nLoaded results for shot {shot_num}")
        print(f"Available keys: {list(results.keys())}")
        
        # Extract density data
        if "density_data" in results:
            density_data = results["density_data"]
            print(f"\nDensity data type: {type(density_data)}")
            if isinstance(density_data, pd.DataFrame):
                print(f"Density DataFrame shape: {density_data.shape}")
                print(f"Density columns: {list(density_data.columns)}")
                print(f"\nFirst few rows:")
                print(density_data.head())
        else:
            print("\nNo density_data found in results")
            density_data = None
    else:
        print("Failed to load results")
        density_data = None
else:
    print("No HDF5 files found. Please run main_analysis.py first.")
    density_data = None


## Step 3: Load Raw Signal Data to Extract Probe Amplitudes


In [4]:
# Load raw signals from HDF5 to extract probe amplitudes
if density_data is not None and h5_files:
    import h5py
    
    with h5py.File(test_h5_file, "r") as f:
        print("\nHDF5 file structure:")
        def print_structure(name, obj):
            print(f"  {name}: {type(obj).__name__}")
        f.visititems(print_structure)
        
        # Try to load signals from HDF5
        probe_amplitudes = {}
        
        if "signals" in f:
            signals_group = f["signals"]
            print("\nAvailable signal groups:")
            for freq_group in signals_group.keys():
                print(f"  - {freq_group}")
                freq_group_obj = signals_group[freq_group]
                
                # Extract probe column names from density data
                if isinstance(density_data, pd.DataFrame):
                    for density_col in density_data.columns:
                        # Try to find matching signal column
                        # Density column format: ne_CH1_45821_056
                        # Signal column format: CH1_45821_056 or CH1_45821_056.csv
                        if "ne_" in density_col:
                            # Extract probe name (e.g., CH1 from ne_CH1_45821_056)
                            probe_name = density_col.replace("ne_", "").split("_")[0]
                            file_suffix = "_".join(density_col.replace("ne_", "").split("_")[1:])
                            
                            # Try different column name formats
                            possible_col_names = [
                                f"{probe_name}_{file_suffix}",
                                f"{probe_name}_{file_suffix}.csv",
                                probe_name,
                            ]
                            
                            for col_name in possible_col_names:
                                if col_name in freq_group_obj:
                                    signal_data = freq_group_obj[col_name][:]
                                    # Calculate amplitude (absolute value)
                                    probe_amplitudes[density_col] = np.abs(signal_data)
                                    print(f"    Found amplitude for {density_col} from {col_name}")
                                    break
        
        print(f"\nExtracted {len(probe_amplitudes)} probe amplitudes")
        if probe_amplitudes:
            for col_name, amp_array in list(probe_amplitudes.items())[:3]:
                print(f"  {col_name}: shape={amp_array.shape}, min={amp_array.min():.4f}, max={amp_array.max():.4f}")
else:
    print("Skipping amplitude extraction (no density data)")
    probe_amplitudes = {}


## Step 4: Test Basic Density Plot (Without Amplitude Coloring)


In [5]:
if density_data is not None:
    plotter = Plotter()
    
    # Extract time from index
    time_data = density_data.index.values if hasattr(density_data, "index") else None
    
    print("Creating basic density plot (no amplitude coloring)...")
    fig, ax = plotter.plot_density(
        density_data,
        time_data=time_data,
        title="Density Plot (Basic)",
        show_plot=True,
        color_by_amplitude=False,
    )
    print("Basic plot created successfully")
else:
    print("Skipping basic plot (no density data)")


## Step 5: Test Density Plot with Amplitude-Based Color Coding


In [ ]:
if density_data is not None and probe_amplitudes:
    print("Creating density plot with amplitude-based color coding...")
    
    # Test with different colormaps
    colormaps = ["coolwarm", "plasma", "viridis", "coolwarm"]
    
    for cmap in colormaps:
        print(f"\nTesting colormap: {cmap}")
        try:
            fig, ax = plotter.plot_density(
                density_data,
                time_data=time_data,
                title=f"Density Plot (Amplitude-Colored, {cmap})",
                show_plot=True,
                probe_amplitude=probe_amplitudes,
                color_by_amplitude=True,
                amplitude_colormap=cmap,
                amplitude_impedance=50.0,
                amplitude_dbm=False,  # Use absolute amplitude, not dB
            )
            print(f"  ✓ {cmap} colormap: Success")
        except Exception as e:
            print(f"  ✗ {cmap} colormap: Failed - {e}")
else:
    print("Skipping amplitude-colored plot (no density data or probe amplitudes)")


## Step 6: Alternative: Load from NAS_DB Cache (If HDF5 Not Available)


In [8]:
# Alternative approach: Load directly from NAS_DB cache
if density_data is None:
    print("Attempting to load from NAS_DB cache...")
    
    try:
        nas_db = NAS_DB(config_path="ifi/config.ini")
        
        # Try to load a known shot (e.g., 45821)
        test_shot = 45821
        shot_data = nas_db.get_shot_data(f"{test_shot}_056.csv", force_remote=False)
        
        if shot_data:
            print(f"\nLoaded data for shot {test_shot}")
            print(f"Available files: {list(shot_data.keys())}")
            
            # Extract probe signal from first file
            first_file = list(shot_data.keys())[0]
            df = shot_data[first_file]
            
            print(f"\nDataFrame from {first_file}:")
            print(f"  Shape: {df.shape}")
            print(f"  Columns: {list(df.columns)}")
            
            # Create synthetic density data for testing
            if "CH1" in df.columns:
                probe_signal = df["CH1"].values
                probe_amplitude = np.abs(probe_signal)
                
                # Create synthetic density (for testing)
                time_data = df.index.values if hasattr(df, "index") else np.arange(len(probe_signal))
                synthetic_density = pd.DataFrame({
                    "ne_CH1_test": np.random.rand(len(probe_signal)) * 1e18
                }, index=time_data)
                
                print("\nCreating test plot with synthetic density and real amplitude...")
                plotter = Plotter()
                fig, ax = plotter.plot_density(
                    synthetic_density,
                    time_data=time_data,
                    title="Test: Density with Amplitude Coloring (Synthetic Density)",
                    show_plot=True,
                    probe_amplitude={"ne_CH1_test": probe_amplitude},
                    color_by_amplitude=True,
                    amplitude_colormap="coolwarm",
                )
                print("Test plot created successfully")
        else:
            print("No data found in NAS_DB cache")
    except Exception as e:
        print(f"Failed to load from NAS_DB: {e}")
else:
    print("Skipping NAS_DB load (already have density data from HDF5)")


## Summary

This notebook tests:
1. ✓ Loading density data from HDF5 results files
2. ✓ Extracting probe signal amplitudes from raw signal data
3. ✓ Creating basic density plots (without amplitude coloring)
4. ✓ Creating amplitude-colored density plots with different colormaps
5. ✓ Fallback to NAS_DB cache if HDF5 files are not available

### Expected Results
- Basic density plot should show standard line plot
- Amplitude-colored plot should show color-coded lines with colorbar
- Color intensity should correspond to probe signal amplitude
- Different colormaps should produce visually distinct plots
